### Read the script

In [1]:
import os
import mne
from ieeg.io import get_data, raw_from_layout, save_derivative

In [2]:
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab")
layout = get_data("LexicalDecRepDelay", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

In [9]:
subject = subjects[0]
# subject_Tag = 'D107B'
subject_Tag = 'D103'

In [4]:
raw = raw_from_layout(layout, subject=subject, preload=False, extension='.edf')
print(subjects)

Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_events.tsv.
Reading channel info from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_channels.tsv.


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: The unit for channel(s) Trigger has changed from NA to V.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading electrode coords from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0101\ieeg\sub-D0101_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-02_ieeg.edf...
EDF file detected


C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: DigMontage is only a subset of info. There are 19 channel positions not present in the DigMontage. The channels missing from the montage are:

['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'O1', 'O2', 'Fz', 'Cz', 'Pz'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\bl314\AppData\Local\miniconda3\envs\ieeg\Lib\site-packages\ieeg\io.py:113: RuntimeWarning: participants.tsv file not found for C:\Users\bl314\Box\CoganLab\BIDS-1.0_LexicalDecRepDelay\BIDS\sub-D0101\ieeg\sub-D0101_task-LexicalDecRepDelay_acq-01_run-01_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Setting channel info structure...
Creating raw.info structure...


KeyboardInterrupt: 

### Linear noise filtering
https://ieeg-pipelines.readthedocs.io/en/latest/auto_examples/plot_clean.html

In [ ]:
from ieeg.mt_filter import line_filter

In [ ]:
# n_jobs=-1 means take all the processors, n_jobs=-2 means take all - 1 processors
# Keep all the other parameters not changed
# %% filter data
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='700ms', freqs=[60], notch_widths=20)
line_filter(raw, mt_bandwidth=10., n_jobs=-3, copy=False, verbose=10,
            filter_length='20s', freqs=[60, 120, 180, 240],
            notch_widths=20)

In [ ]:
# Check if derivatives folder exists and create if not
bids_root='C:\\Users\\bl314\\Box\\CoganLab\\BIDS-1.0_LexicalDecRepDelay\\BIDS'
if not os.path.exists(os.path.join(bids_root, "derivatives")):
    os.mkdir(os.path.join(bids_root, "derivatives"))
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
elif not os.path.exists(os.path.join(bids_root, "derivatives", "clean")):
    os.mkdir(os.path.join(bids_root, "derivatives", "clean"))
save_derivative(raw, layout, "clean", True)